In [ ]:
# default_exp losses.tacotron2

In [ ]:
# export
from random import choice, randint
from uberduck_ml_dev.models.tacotron2.tacotron2 import Tacotron2
import time
from typing import List

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from uberduck_ml_dev.utils.utils import get_mask_from_lengths
from uberduck_ml_dev.data_loader import TextMelDataset, TextMelCollate

from uberduck_ml_dev.utils.plot import save_figure_to_numpy
from uberduck_ml_dev.utils.utils import reduce_tensor
from uberduck_ml_dev.monitoring.statistics import get_alignment_metrics
import nemo
from typing import NamedTuple
import pdb


class Tacotron2Loss(nn.Module):
    def __init__(
        self,
        pos_weight,
        non_attentive=False,
        location_specific_attention=True,
        sampling_rate=None,
        hop_length=None,
    ):
        if pos_weight is not None:
            self.pos_weight = torch.tensor(pos_weight)
        else:
            self.pos_weight = pos_weight
        self.non_attentive = non_attentive
        self.location_specific_attention = location_specific_attention
        self.sampling_rate = sampling_rate
        self.hop_length = hop_length
        super().__init__()

    def forward(self, model_output: NamedTuple, target: NamedTuple):

        # NOTE (Sam) https://github.com/JoungheeKim/Non-Attentive-Tacotron/blob/c82049be0162f59898daaf8e1fa19fecaadeec31/tacotron/model.py
        # uses mae loss as well on mels
        mel_target = target.mel_padded
        mel_target.requires_grad = False

        mel_out_postnet = model_output.mel_outputs_postnet
        mel_output = model_output.mel_outputs

        if self.location_specific_attention:
            gate_target = target.gate_target
            gate_target.requires_grad = False
            gate_pred = get_mask_from_lengths(
                model_output.gate_pred[:, 0], max_len=gate_target.shape[1]
            ).float()
            gate_loss_batch = nn.BCEWithLogitsLoss(
                pos_weight=self.pos_weight, reduce=False
            )(gate_target, gate_pred).mean(axis=[1])
            gate_loss = torch.mean(gate_loss_batch)

        if self.non_attentive:
            durations_padded = target.durations_padded
            predicted_durations = model_output.predicted_durations
            duration_mask = (durations_padded > 0.0).float()
            non_zero_duration = duration_mask.sum()

            in_seconds = self.sampling_rate / self.hop_length
            predicted_durations = predicted_durations / in_seconds
            durations_padded = durations_padded / in_seconds

            predicted_durations_mse = nn.MSELoss(reduction="none")(
                predicted_durations, durations_padded
            ).sum()
            predicted_durations_mse = (predicted_durations_mse * duration_mask).sum(
                axis=1
            ) / non_zero_duration
            durations_loss_batch = torch.nan_to_num(predicted_durations_mse)
            durations_loss = durations_loss_batch.mean()

        mel_loss_batch = nn.MSELoss(reduction="none")(mel_output, mel_target).mean(
            axis=[1, 2]
        ) + nn.MSELoss(reduction="none")(mel_out_postnet, mel_target).mean(axis=[1, 2])

        mel_loss = mel_loss_batch.mean()

        if self.location_specific_attention:
            loss = mel_loss + gate_loss
            loss_batch = mel_loss_batch + gate_loss_batch
            return (
                loss,
                loss_batch,
                mel_loss,
                mel_loss_batch,
                gate_loss,
                gate_loss_batch,
            )
        if self.non_attentive:
            loss = mel_loss + durations_loss
            loss_batch = mel_loss_batch + durations_loss_batch
            return (
                loss,
                loss_batch,
                mel_loss,
                mel_loss_batch,
                durations_loss,
                durations_loss_batch,
            )

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

